### This Jupyter Notebook presents a methodological template for conducting both Exploratory Data Analysis (EDA) and Predictive Modeling. The techniques are illustrated using an example dataset and are structured for straightforward adaptation to alternative datasets.

The aim of this jupyter book is to demonstrate some of the python tools to perform exploratory data analysis and predictive modeling.
EDA is the phase of analysis aimed at understanding the characteristics of the data and uncovering patterns. It typically precedes predictive modeling. Model building allows us to make predictions and derive insights from data.

### Steps that the code performs:

1. **Data Loading and Cleaning:** The code loads data from an Excel file, cleans the column names, and handles missing values. It also removes duplicate rows and columns, as well as those with a unique value.
2. **Feature Creation and Type Changing:** The code creates a new feature called Qv_MJ_l and converts the data types of some columns to numeric if they aren't already. It transforms a categorical variable into a numerical one and selects the numeric columns for further analysis.
3. **Data Analysis and Visualization:** The code calculates basic statistics and standardised values for the numeric columns. It looks for outliers using three standard deviations and the interquartile range (IQR). It computes the correlation matrix between variables and visualises it using a heatmap. It generates distribution plots and box plots for key parameters. It analyses trends and relationships between variables using scatter plots and pair plots.
4. **Summary of Findings:** The code generates a summary table of key metrics and saves the processed data to a new Excel file.
5. **Data Processing:** The code performs a classical linear regression with statsmodels. It selects features and the target variable, adds a constant term for the intercept, and fits the model. It draws a y-ypredict scatterplot and checks the variance inflation factor (VIF). It removes some features and fits a new model. It creates regression plots and checks the VIF.
6. **Stepwise Regression Technique:** The code utilises a function to add and select features by steps according to AIC/BIC criteria. It fits the model to the selected features and draws a y-ypredict scatterplot and checks the VIF.
7. **Linear Regression using scikit_learn and Cross-Validation (cv):** The code loads additional libraries and instantiates LinearRegression. It defines initial parameters and splits data into training and testing sets. It uses RFECV for recursive feature elimination with cross-validation. It checks the results to select features.
8. **Comparing Results:** The code defines a function to calculate rmse from cv_test (train) and test: Linear Regression. It calculates the results of linear regression, lasso regression and ridge regression.
9. **Principal Component Regression (PCR) and Cross-Validation:** The code attempts PCA analysis for regression of principal components. It calculates principal components and displays feature weights of each principal component. It checks the most important features for each principal component. It calculates RMSE using KFold and looks for the best option. It selects the number of principal components and calculates train and test rmse. It calculates R2 score and draws the residuals scatterplot.
10. **Partial Least Squares (PLS) regression:** The code utilises PLS regression with cross-validation. It looks for the best number of components and displays results. It checks R2 score for the best estimator. It utilises a function for sequential feature/component elimination. It displays the result with the best score and checks other good solutions.
11. **Random Forest Regression:** The code utilises Random Forest Regression with cross-validation. It looks for the best hyperparameters using GridSearchCV. It evaluates the model using RMSE and R2. It plots feature importances. It displays the scatterplot of the prediction. It calculates the OOB (Out-of-Bag) score.
12. **Shap (SHapley Additive exPlanations):**  The code utilises SHAP to explain the output of Random Forest Regression model.
13. **Visualising graphs with the graphviz library:** The code imports necessary libraries and selects a tree to plot. It creates the graph using export_graphviz and displays it.


The database employed to illustrate the tools and techniques presented in this Jupyter Notebook is derived from the distinguished article: <i>Chemical Descriptors for a Large-Scale Study on Drop-Weight Impact Sensitivity of High Explosives</i>. This study investigates the relationship between the results of the drop-weight impact test—used to evaluate the handling sensitivity of high explosives—and a compendium of molecular and chemical descriptors associated with the explosives under examination.

<b>Frank W. Marrs, Jack V. Davis, Alexandra C. Burch, Geoffrey W. Brown, Nicholas Lease, Patricia L. Huestis, Marc J. Cawkwell, and Virginia W. Manner, 2023.</b>
<i>Chemical Descriptors for a Large-Scale Study on Drop-Weight Impact Sensitivity of High Explosives</i>. <i>Journal of Chemical Information and Modeling.</i><br>
https://pubs.acs.org/doi/10.1021/acs.jcim.2c01154<br>
<br>

**DISCLAIMER:** This code is provided for educational and demonstrative purposes only. Its sole objective is to illustrate Python techniques for data visualisation and analysis. The datasets used in the examples serve purely as illustrative material; no comprehensive or contextual analysis of these specific datasets has been undertaken or is implied. The primary focus remains on the implementation of technical methodologies, rather than the in-depth interpretation of the data itself.
For the purposes of this notebook, minor modifications have been introduced into the database in order to facilitate the illustration of certain techniques presented herein.

<i>**Fernando García Bastante<br>
Universidade de Vigo**</i>
# .........................................................................................................................................

# Setup and Display Options
## The first cell of the Jupyter notebook performs some initial setup and imports necessary libraries
https://numpy.org/<br>
https://pandas.pydata.org/<br>
https://matplotlib.org/<br>
https://seaborn.pydata.org/

In [ ]:
# ---- Load libraries for data manipulation and visualisation
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load libraries for data rendering as HTML (optional)
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Configure pandas and NumPy to enhance DataFrame and array output
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 1500
pd.options.display.max_columns = 300
pd.set_option('display.expand_frame_repr', False)
np.set_printoptions(precision=3)

# Excel Data Loading with Error Handling

In [ ]:
# ---- Load Dataset from Excel with Error Handling ----
file_path = "data_.xlsx"
sheet_name = "datasheet"

try:
    # Skip the first row which may contain metadata or headers
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=1)
    print("✅ Data successfully loaded.")
except FileNotFoundError:
    print(f"❌ Error: The file '{file_path}' was not found.")
    df = None
except Exception as e:
    print(f"❌ An unexpected error occurred while loading the file: {e}")
    df = None

# Initial Data Display and Inspection

In [ ]:
# ---- Display the first 5 rows of the DataFrame
df.tail()

In [ ]:
# ---- Display information about the DataFrame (rows, columns, data types)
print(df.info())

In [ ]:
# ---- Display only the names and data types of each column (floats and objects (strings))
print("Data Types of Each Column:")
print(df.dtypes) # Data type summary
print(df.shape) # (rows, columns)

In [ ]:
# ----A preliminary overview of the statistics of the target variable
df['log H50'].describe()

In [ ]:
# ----Check the variability of the log H50 measurements for some "individual" explosives -> according Chem_Formula
df_target_all = df.groupby('Chem_Formula').agg({'log H50': ['count', 'mean', 'std']})  # Get the count, mean and standard deviation for each explosive
df_target_all[df_target_all['log H50']['count']>40]  #  Get the mean and std deviation from explosives with a high number of data

# Cleaning Column Names

In [ ]:
# -----Clean column names for easier access and scripting ----
# Removes leading/trailing spaces, replaces special characters
df.columns = (
    df.columns
      .str.strip()  # Removes any leading, and trailing whitespaces
      .str.replace(' ', '_')
      .str.replace(r'[(){}\[\]]', '', regex=True)
)
# For stripping all non-alphanumeric characters:
# df.columns = df.columns.str.replace(r'[^\w]', '', regex=True)
print("🧼 Cleaned Column Names:")
print([names for names in df.columns])

# Filtering Data: Handling Nulls, Duplicates, Unique-Value Columns, and Irrelevant Fields

In [ ]:
# ----Remove rows (the last ones) and columns (the last one) of NaN's
df.dropna(axis='rows', inplace= True, how='all')
df.dropna(axis='columns', inplace= True, how='all')
df.tail()

In [ ]:
# ----Only get the data with 'threshold = NaN' and 'group = [NCOO, NOOO, NNOO]
df = df[pd.isna(df['threshold'])]
df = df[df['group'].isin(['NCOO', 'NOOO', 'NNOO'])]
print('Now the dataframe is', df.shape, 'rows x columns')

In [ ]:
# ----Delete colums of no interest in this exercise
df.drop(['id', 'threshold', 'Method', 'reference', 'lab', 'grit'], inplace=True, axis=1)

In [ ]:
# ----Remove duplicated rows: this assumes that the database contains repeated-duplicated values for some tests
# If you think all the data is OK comment this code
df.drop_duplicates(inplace=True)
df[df.duplicated(keep=False)] # checking duplicates again

In [ ]:
# ----Check the number of rows and columns after removing duplicates
print('Now the dataframe is', df.shape, 'rows x columns')

In [ ]:
# ----Select columns with only one unique value (dummy columns)
dummy_col = df.nunique()==1  # Get the column names of unique values
dummy_col = dummy_col[dummy_col].index.tolist()  # Get the column names of unique values
print(dummy_col)  # Display the dummy columns -you can remove them from df dataframe but we will keep them for now for teaching another way to select them

In [ ]:
# ----Other option: check for columns with a known unique value (0, 1 or "yes" in this example)
no_dummy_col  = (~df.isin([0,1,"yes"])).any(axis=0)  # Identify columns with more than one unique value
no_dummy_col.tail(10)

In [ ]:
# ----Remove the "False" columns (unique value columns) to keep only informative columns
df = df.loc[:, no_dummy_col]
df.tail()

In [ ]:
# ----Replace empty cells in columns with NaN missing values for consistency
df = df.replace(' ', np.nan)

In [ ]:
# ----Count missing values per column
missing_data_count = df.isnull().sum()
missing_data_percent = 100 * df.isnull().mean()  # Values in percentage

# Joining the two series
df_ = pd.DataFrame({'missing_data_count': missing_data_count, 'missing_data_percent': missing_data_percent})
print(df_[df_['missing_data_count']>0])

In [ ]:
# ----Fix the missing data in Chem_Formula
# Find the NaN Values in Chem_Formula
df[df['Chem_Formula'].isnull()]

In [ ]:
# ----Calculate your owns formulas from C, H, N, O and Cl columns in the database (easier option in this example)
df["Chem_Formula"] = pd.DataFrame('C' + df["C"].astype(int).astype(str) +
                                  'H' + df["H"].astype(int).astype(str) +
                                  'N' + df["N"].astype(int).astype(str) +
                                  'O' + df["O"].astype(int).astype(str) +
                                  'Cl'+ df["Cl"].astype(int).astype(str))

In [ ]:
# ----Impute missing values in the 'C_v_J_g_K ' column using the mean of the 'C_v_J_g_K' values grouped by 'group'
df['C_v_J_g_K'] = df['C_v_J_g_K'].fillna(df.groupby('group')['C_v_J_g_K'].transform('mean'))
# df = df.dropna(subset=["C_v_J_g_K', "Chem_Name"], how="any") # in this case you are removing the rows with NaN values in C_v_J_g_K and Chem_Name columns

In [ ]:
# ----Checking the number of log_H50 values for each explosive
target = 'log_H50'  # log_H50 is the target variable

unique_explosives_ = df.groupby('DerivName')[target].count()
unique_explosives__ = df.groupby('Chem_Name')[target].count()
unique_explosives___ = df.groupby('Chem_Formula')[target].count()
print('Number of unique explosives according to DerivName, Chem_Name, Chem_Formula:', unique_explosives_.shape[0], unique_explosives__.shape[0], unique_explosives___.shape[0])
print('Explosives with several log_H50 measures according to Chem_Formula')
print(unique_explosives___[unique_explosives___>1])  # Perhaps there are no different explosives in the database that have the same Chem_Formula but you must check it

In [ ]:
# ----Display duplicated Chem_Formula values to check if there are different explosives but the same formula (manual labor)
df[df['Chem_Formula'].duplicated(keep=False)].sort_values(['Chem_Formula', 'Chem_Name'])  # keep=False shows all duplicated rows

In [ ]:
# ----Rename Chem_Formula for different explosives with same formula Example: the 2,3,4_TNT (index=71) and the 2,4,6_TNT (there are a lot of it) have the same formula.
# Change the formula of the index 71 adding 'bis':
df.loc[71, 'Chem_Formula'] = 'bis' + df.loc[71, 'Chem_Formula']
# Now for 2,4,6-Trinitrobenzylalcohol
df.loc[120, 'Chem_Formula'] = 'bis' + df.loc[120, 'Chem_Formula']
#.................................Do it yourself_________________________________

In [ ]:
# ----Check the variability of the log_H50 measurements for individual explosives
df_target_st = df.groupby('Chem_Formula').agg({target: ['count', 'mean', 'std']})  # Get the count, mean and standard deviation for each explosive
df_target_st[df_target_st[target]['count']>25]  #  Get the mean and std deviation from explosives with a high number of data

# Creating New Features and Changing Data Types

In [ ]:
# ----Add a new feature
df["q_per_mol"] = df["q_per_g"]/df["gas_moles_per_g"]
# Change the type to string to show later how to convert strings to numbers
df.q_per_mol = df.q_per_mol.astype('string') # Convert column 'q_per_mol' to string type
df["q_per_mol"].head()

In [ ]:
# ----Display the data types of each column
print("Data Types of Each Column:")
print(df.dtypes)
print(df.shape) # (rows, columns)

In [ ]:
# ----Case you need to convert relevant columns to numeric if they aren't already
numeric_convert_columns = ["q_per_mol"] # only this column in this example
df[numeric_convert_columns] = df[numeric_convert_columns].apply(pd.to_numeric, errors='coerce', downcast="float") # coerce invalid parsing will be set as NaN
df["q_per_mol"].dtypes

In [ ]:
# ----Surely be this one alternative a better option:
df["q_per_mol"] = df["q_per_mol"].values.astype(float)  # Convert 'q_per_mol' to float type
df["q_per_mol"].dtypes

In [ ]:
# ---- Define your own new feature
#df[''] = df['']
#df[''].describe()

#  Encoding Categorical Variables

In [ ]:
# ----Label Encoding: transform a categorical variable (group) into numeric using factorize function
df['num_type'], uniques = pd.factorize(df['group']) # Create a numerical representation of the 'group' column
df['num_type'].head(10) # Display the first 15 values of the 'num_type' column

In [ ]:
# ----Display the Type variable (num_type 0:NOOO, num_type 1:NCOO...)
uniques

In [ ]:
# ----Custom mapping: it is appropriate to map according to some ordinal criteria, e.g. from lower to higher average sensitivity of the functional group
custom_map = {'NCOO': 1, 'NNOO': 0, 'NOOO': -1}
df['num_type'] = df['group'].map(custom_map)

In [ ]:
# ----Using one hot encoding is other option
my_groups = pd.get_dummies(df['group'])  # Only need n-1 codes: pd.get_dummies(df['group'], drop_first=True)
df_ = pd.concat([df, my_groups], axis=1)
df_[uniques].head(10)  # Note that there are different columns with repeated names

# Basic Statistics

In [ ]:
# ----Basic statistics with pandas
df.describe() # Explore the variant: df.describe(include='all')

In [ ]:
# ----In case you need only the mean values...
print('----------------Average values')
df.mean(numeric_only=True)

In [ ]:
# ----Checking the number of lnH50 values for each group
print('------------- All values: there are several repeated explosives ------------')
print('------ data number by', df.groupby('group')[target].count())
print('------ mean value by', df.groupby('group')[target].mean())
print('------ standard value by', df.groupby('group')[target].std())
print('------ aprox. number of unique explosives (according Chem_Formula) by', df.groupby('group')['Chem_Formula'].nunique())

In [ ]:
# ----Get the numeric and the "categorical" features names
num_columns = [col for col in df.select_dtypes(include=[np.number]).columns]
cat_columns = [col for col in df.select_dtypes(include=[object]).columns]
print(num_columns)
print(cat_columns)

In [ ]:
# ----Make a dataframe with the numerical columns
num_df=df[num_columns]
num_df.tail()

In [ ]:
# ----Checking the types
print(num_df.dtypes.tail(10))

## Standardised Features, Search for Outliers and Aggregated Data

In [ ]:
# ----Standardized values
std_num = (num_df - num_df.mean()) / num_df.std()  # Calculate the standarised values
std_df = pd.concat([std_num, df[cat_columns]], axis=1)  # Add the non-numeric columns
print(std_df)

In [ ]:
# ----Check the variability of the standardised target measurements
print(' Data for standardised values of target')
std_df[target].describe()

In [ ]:
# ----Check for outliers in numeric columns [later we will draw boxplots to get the outliers graphically]
# An outlier is defined as an observation that deviates significantly from other values in a dataset, which does not inherently indicate an error or incorrectness

# Method a- with 3 standard deviations in this example]
std_dev_limit = 3 # Set the standard deviation limit for outlier detection
df_outlier = std_df[std_df[num_columns].abs()>std_dev_limit] # Identify outliers based on standard deviation only in the [num_columns]

# Number of ouliers in each column (NaN's in df_outlier dataframe are non-outlier values)
df_outlier.count()

In [ ]:
# ---- Method b- with 1.5 IQR
def check_outliers(column, df=df):
    """
    Identifies outliers in a given column using the Interquartile Range (IQR) method.
    Args:
        column (str): The name of the column to check for outliers.
    Returns:
        pandas.DataFrame: A DataFrame containing the rows identified as outliers.
    """
    Q1 = df[column].quantile(0.25) # Calculate the first quartile
    Q3 = df[column].quantile(0.75) # Calculate the third quartile
    IQR = Q3 - Q1 # Calculate the third quartile
    lower_bound = Q1 - 1.5 * IQR # Calculate the lower bound for outliers
    upper_bound = Q3 + 1.5 * IQR # Calculate the upper bound for outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]  # Identify outliers
    return outliers
# Check the outliers for each column in the dataframe
for col in num_columns:
    outliers = check_outliers(col)
    if not outliers.empty:
        print(f"\nOutliers in {col}:")
        print(outliers["Chem_Formula"]) # Print the 'Chem_formula' of outliers

In [ ]:
# ----Check the variability of the standardised target measurements for individual explosives
std_group_target = std_df.groupby('Chem_Formula').agg({target: ['count', 'mean', 'std']})  # Get the count, mean and deviation for each explosive
print('Calculated from standardised values')
std_group_target[std_group_target[target]['count']>25]  #  Get the std deviation only from a high number of data

In [ ]:
# ----Let's group the df original dataframe by 'Chem_Formula' using the mean of log_H50 as the target variable
#  std_df[target] = df[target]  # Works with log_H50 to facilitate interpretation
dic_column_names = {col: 'first' for col in df.columns.drop('Chem_Formula')}  # Get the columns names and operator to apply for each (save the first value)
dic_column_names[target] = 'mean'  # Change the operator to mean for target column
u_df = df.groupby('Chem_Formula').agg(dic_column_names).reset_index()  #  Create the new datafrmae with unique explosives
print(u_df.shape)

In [ ]:
# Basic statistics of grouped data by explosive (non_standardised data)
u_df.describe()

In [ ]:
# ----Check the variability of log_H50 measurements for each group (non-standardised)
df_target = u_df.groupby('group').agg({target: ['count', 'mean', 'std']})  # Get the count, mean and standard deviation for each group
df_target[df_target[target]['count']>25]  #  Get the std deviation from a high number of data

In [ ]:
# ----Standardized values except log_H50 and num_type
u_std_df = (u_df[num_columns] - u_df[num_columns].mean()) / u_df[num_columns].std()
u_std_df[target] = u_df[target]  # Works with log_H50 instead of stardrdised values to facilitate interpretation
u_std_df['num_type'] = u_df['num_type']
u_std_df = pd.concat([u_std_df, u_df[cat_columns]], axis=1)  # Add the non-numeric columns
u_std_df.describe(include='all')

## Correlations between features

### The target is lnH50

In [ ]:
# ----Get a dataframe with the most correlated features with the variable 'lnH50'
def analyze_imp_correlations(df, number):
    """
    Analyses and selects the top correlated features with the target variable 'lnH50'.
    Args:
        df (pandas.DataFrame): The input DataFrame.
        number (int): The number of top correlated features to select.
    Returns:
        pandas.DataFrame: A DataFrame containing the selected features and the target variable.
    """
    imp_corr = df.corr(numeric_only=True)[target].abs().sort_values(ascending=False) # Calculate correlations with 'ln_H50'
    top_correlated_vars = imp_corr[1:number].index.tolist() # Select top correlated features
    top_correlated_vars.insert(0, target) # Add 'lnH50' to the list
    top_correlated_vars.extend(['group', 'Chem_Formula']) # insert the num_type feature to the end of the list
    # top_correlated_vars.remove('H50') # remove the H50 feature
    return df[top_correlated_vars] # Return a DataFrame with selected features

In [ ]:
# ----Choose the number of most correlated features to analyze and apply 'analyze_imp_correlations(df, number)'
max_number_correlated = 15
correlated_df = analyze_imp_correlations(u_std_df, max_number_correlated+1) # Get the DataFrame with top correlated features
print(f"\nDataframe with the {max_number_correlated} most correlated features with the target variable lnH50")
print(correlated_df.head(3))
print('......................................................................................................................')

In [ ]:
# ----Correlation Analysis: compute correlation matrix for key parameters
correlation_matrix = correlated_df.corr(numeric_only=True)

# Print the correlation matrix
print("\nCorrelation Matrix:")
print(correlation_matrix)

In [ ]:
# ----Visualize the correlation matrix using a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True) # Create a heatmap of the correlation matrix
plt.title("Correlation Heatmap")
plt.show() # Display the heatmap

## Distribution Plots of Key Variables

In [ ]:
# ----Distribution plots of key parameters
number_keys = 5  #  Select the number of key parameters to plot
for param in correlated_df.columns[0:number_keys]:
    plt.figure(figsize=(8, 5))
    sns.histplot(u_std_df[param], kde=True, bins=20, color="blue") # Note that this example use standardized data
    plt.title(f"Distribution of standardized {param}")
    plt.xlabel(param)
    plt.ylabel("Frequency")
    plt.show()

## Box plots to visualise the dispersion of key parameters and identify possible outliers

In [ ]:
# ----Boxplots for key parameters to visualize spread and identify potential outliers
for param in correlated_df.columns[0:number_keys+1]:
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=correlated_df, x=param) # Note that this example use correlated_df (standardized) data
    if param != 'log_H50':
        plt.title(f"Boxplot of {param} standardized")
    else:
        plt.title(f"Boxplot of {param}")
    plt.xlabel(param)
    plt.show()

## Trend Analysis to visualise correlations

In [ ]:
# ----Plot trends of `log_H50` vs features in correlated_df
for param in correlated_df.columns[1:number_keys+1]:
    plt.figure(figsize=(8, 5))
    sns.scatterplot(data=correlated_df, x=param, y=target, hue='q_per_g', size='group', palette='viridis') #Note that standardised data are used here
    plt.xlabel(param)
    plt.ylabel(target)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Compound legend")
    plt.tight_layout()
    plt.show()

In [ ]:
# ----Pairplot for visualizing relationships between key features
sns.pairplot(data=correlated_df, diag_kind='kde', hue='group', palette='Set2')
plt.suptitle("Pairplot of Key Parameters", y=1.02)
plt.show()

## Summary of findings, saving results

In [ ]:
# ----Select the features to save
feat_select = ['Chem_Name'] + [i for i in  correlated_df.columns[0:4]] + ['group']

# Creating tables for saving
summary_table_low_imp = u_std_df[feat_select].sort_values(by=target, ascending=True).head(10)  # Dataframe for compounds with high impact sensitivity
summary_table_high_imp = u_std_df[feat_select].sort_values(by=target, ascending=False).head(10)  # Dataframe for compounds with low impact sensitivity

# Display the tables
print("\nTop 10 Compounds with the Highest Impact Sensitivity (`log_H50`):")
print(summary_table_low_imp)
print("\nTop 10 Compounds with the Lowest Impact Sensitivity (`log_H50`):")
print(summary_table_high_imp)

# Save processed data to a new Excel file
output_file_path = "processed_data_analysis.xlsx"
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name="Processed_Data", index=False)  # Save processed data to a sheet named 'Processed_Data'
    summary_table_low_imp.to_excel(writer, sheet_name="Summary_Table_Low_Imp", index=False)  # Save high impact sensitivity data to a separate sheet
    summary_table_high_imp.to_excel(writer, sheet_name="Summary_Table_High_Imp", index=False)  # Save low impact sensitivity data to a separate sheet

print(f"\nProcessed data saved to {output_file_path}")

# Data Processing

## Classical Linear Regression
https://www.statsmodels.org/dev/index.html

In [ ]:
# ----Linear regression fitting with statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor # for checking correlation effects betwen features
from statsmodels.tools.eval_measures import rmse # for getting the root mean squared error
from statsmodels.stats.dist_dependence_measures import distance_correlation  # for calculate the distance correlation

In [ ]:
# ----Getting the features and the target variable
X_features = correlated_df.drop([target, 'group', 'Chem_Formula'], axis = 1) # Only the numeric values
y = correlated_df[target]

### Before fitting the model, we are going to calculate the distance_correlation between features and the target variable

In [ ]:
# ----Calculate the distance correlation for each feature with the target variable (best than Pearson for non linear correlation)
distance_correlations = {}
for col in X_features.columns:
    d_corr = distance_correlation(X_features[col], y)
    distance_correlations[col] = d_corr

# Sort the features by distance correlation in descending order
sorted_distance_correlations = sorted(distance_correlations.items(), key=lambda item: item[1], reverse=True)

# Extract feature names and their distance correlations for plotting
features = [item[0] for item in sorted_distance_correlations]
dCorrelations = [item[1] for item in sorted_distance_correlations]

# Calculate the distance correlation using all the features with the target variable
dCorr_all_features = distance_correlation(X_features, y)
print('the distance correlation calculated using all features is', f'{dCorr_all_features:0.2f}')

# Create a bar plot of the distance correlations
plt.figure(figsize=(8, 5))
ax = sns.barplot(x=dCorrelations, y=features)
plt.title(f'Distance Correlation with {target}')
plt.xlabel('Distance Correlation')
plt.ylabel('Features')

# Add the distance correlation values as text labels on the bars
for index, value in enumerate(dCorrelations):
    ax.text(value, index, f'{value:.2f}', va='center') # Using ax.text to place text

plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()

In [ ]:
# ----Checking the features combination with the highest distance correlation with the target variable
import itertools
feat_number = 4 # Number of features to combine
combi = list(itertools.combinations(features[0:15], feat_number))
dist_corr = [(select_features, distance_correlation(X_features[list(select_features)], y)) for select_features in combi]  # Calculate the distance correlation for each pair of features
dist_corr = pd.DataFrame(dist_corr, columns=['features', 'distance_correlation']).sort_values('distance_correlation', ascending=False)
print(dist_corr.head(5))

In [ ]:
# ----Begin the regression analysis
# Add a constant term to X_features for the intercept
X0 = sm.add_constant(X_features)

In [ ]:
print(X0.head())

In [ ]:
# ----Fit the model
model = sm.OLS(y, X0).fit()  # Fit an Ordinary Least Squares (OLS) regression model
print(model.summary(alpha = 0.05))  # Print the model summary
print(f'The rmse is: {rmse(y, model.predict(X0)):.2f}') # Print the root mean squared error

In [ ]:
# ----Function for drawing a y-ypredict scatterplot
def plot_scatter (y_predict, y=y):
    """
    Creates a scatterplot of predicted values (y_predict) against actual values (y).
    Args:
        y_predict (array-like): Predicted values.
        y (array-like, optional): Actual values. Defaults to the global 'y' variable.
    """
    residuals = np.abs(y-y_predict)  # Calculate residuals
    threshold = 3 * np.std(residuals)  # setting a threshold to identify possible outliers
    fig = plt.figure(figsize=(8, 5))
    ax = fig.add_subplot()
    ax.set_aspect('equal')
    sns.scatterplot(x=y_predict, y=y)  # Create a scatterplot
    sns.scatterplot(x=y_predict[abs(residuals) > threshold], y=y[residuals > threshold], color='red', label='Outliers?')  # Highlight potential outliers
    xmin, xmax = y_predict.min() -.25, y_predict.max() +.25
    sns.lineplot(x=[xmin, xmax], y=[xmin, xmax], color='green')
    plt.title(f"Impact Sensitivity prediction")
    plt.xlabel('y_predict')
    plt.ylabel("y")
    plt.tight_layout()
    plt.show()
    print('Pearson correlation matrix:')
    print(np.corrcoef(y, y_predict))  # Print the Pearson correlation matrix
    print(f'The rmse is: {rmse(y, y_predict):.2f}')  # Print the rmse

In [ ]:
# ----Create a scatterplot of predicted vs actual values
y0_predict = model.predict(X0)
plot_scatter(y0_predict, y)

In [ ]:
# ----Function for drawing a residuals Box-Plot
def plot_residuals_by_class(df, residuals_col, group_col=None, title="Residuals Box Plots",
                            xlabel=None, ylabel="Residuals"):
    """
    Generates boxplots of residuals, either grouped by a specified column or
    as a single boxplot for all residuals if no group column is provided.
    Calculates and displays the RMSE for each group (or overall) directly on the plot.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the residual values and optionally the group column.
    residuals_col : str
        The name of the column in 'df' that contains the residual values.
    group_col : str, optional
        The name of the column in 'df' that contains the group (class) labels.
        If None, a single boxplot for all residuals will be generated. Defaults to None.
    title : str, optional
        Main title for the plot. Defaults to "Residuals Boxplots".
    xlabel : str, optional
        Label for the X-axis. If 'group_col' is provided, defaults to the group_col name.
        If 'group_col' is None, it won't be displayed. Defaults to None.
    ylabel : str, optional
        Label for the Y-axis. Defaults to "Residuals".
    """

    if residuals_col not in df.columns:
        raise ValueError(f"Column '{residuals_col}' not found in the DataFrame.")

    # --- Calculate RMSE ---
    if group_col:
        rmse_values = df.groupby(group_col)[residuals_col].apply(lambda x: np.sqrt(np.mean(x**2)))
        print("RMSE per Group:")
        [print(f"  Group {name}: {value:.3f}") for name, value in rmse_values.items()]

    # Calculate overall RMSE
    overall_rmse = np.sqrt(np.mean(df[residuals_col]**2))
    all_rmse_values = pd.Series({'All Data': overall_rmse}) # Wrap in Series for consistent handling
    print(f"Overall RMSE: {overall_rmse:.3f}")

    # --- Plotting Setup ---
    sns.set_style("whitegrid")
    plt.figure(figsize=(8, 5))

    # --- Create Boxplots ---
    if group_col:
        sns.boxplot(hue=group_col, y=residuals_col, data=df, hue_order=rmse_values.index.tolist(), notch=True, gap=.1)
        # Default xlabel to group_col name if not explicitly set
        if xlabel is None:
            xlabel = group_col
    else:
        # Single boxplot for all data
        sns.boxplot(y=residuals_col, data=df, flierprops=dict(markerfacecolor='0.7', markersize=5))
        # No xlabel needed for a single boxplot
        if xlabel is None:
            xlabel = "" # Empty string to remove label

    # --- Add Titles and Labels ---
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.yticks(fontsize=10)

    # --- Annotate with RMSE values ---
    if group_col:
        for i, group_name in enumerate(rmse_values.index):       
            rmse_value = rmse_values[group_name]
            plt.text(-0.3+0.3*i, - 1.2, f'RMSE: {rmse_value:.3f}',
                 horizontalalignment='center', color='black', weight='semibold', fontsize=14)
    else:
        # 'x' position for a single boxplot is 0
        plt.text(0, -1.2, f'Overall RMSE: {overall_rmse:.3f}',
                 horizontalalignment='center', color='black', weight='semibold', fontsize=12)

    # --- Final Touches ---
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
# Residuals Box-Plot
u_std_df['residuals_0'] = y-y0_predict  # Add the residuals to the unique explosives dataframe 
plot_residuals_by_class(u_std_df, residuals_col='residuals_0', group_col='group')

In [ ]:
# ----Checking variance inflation factor (VIF < 5(10))
pd.Series([variance_inflation_factor(X0.values, i) for i in range(X0.shape[1])], index=X0.columns)  # Calculate and display VIF for each feature

In [ ]:
# Removing some features and checking VIF again
features_to_remove = ['NOO',
                     'Atom_E_atom',
                     'remain_O2',
                     'q_per_mol',
                     'NO',
                     'Moment2',
                     'NO_group'
                      ]
X1 = X0.drop(features_to_remove, axis = 1) # New X dataframe
pd.Series([variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])], index=X1.columns)

In [ ]:
# Fitting the new model
model1 = sm.OLS(y, X1).fit()  # Fit a new OLS regression model
print(model1.summary(alpha = 0.05))

In [ ]:
# Removing the const... features and Fitting the new model
X1 = X1.drop(['gas_C', 'gas_CO2'], axis = 1)
model1 = sm.OLS(y, X1).fit()
print(model1.summary(alpha = 0.05))  # Print the model summary

In [ ]:
# Drawing the sscatterplot y-y_predict
y1_predict = model1.predict(X1)
plot_scatter(y1_predict, y)

In [ ]:
# Residuals Box-Plot
u_std_df['residuals_1'] = y-y1_predict  # Add the residuals to the unique explosives dataframe 
plot_residuals_by_class(u_std_df, residuals_col='residuals_1', group_col='group')

In [ ]:
# Creating regression plots for visualizing the relationship between features and the target variable
fig = plt.figure(figsize=(14, 8))
fig = sm.graphics.plot_regress_exog(model1, 'q_per_g', fig=fig)  # Create a regression plot for the 'Q' feature

In [ ]:
# Creating regression plots
fig = plt.figure(figsize=(14, 8))
fig = sm.graphics.plot_regress_exog(model1, 'Moment1', fig=fig)  # Create a regression plot for the 'Cs' feature

In [ ]:
# Checking variance inflation factor (VIF < 5(10)) to assess multicollinearity
pd.Series([variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])], index=X1.columns)

## Stepwise regression technique

In [ ]:
# Function for adding and selecting features by steps according to aic/bic criteria
def stepwise_selection(X, y):
    """
    Performs stepwise feature selection using AIC/BIC criteria.
    Args:
        X (pandas.DataFrame): The feature matrix.
        y (pandas.Series): The target variable.
        p_value (float, optional): The significance level for feature selection. Defaults to 0.05.
    Returns:
        list: A list of selected features.
    """
    features = list(X.columns)  # Get the list of all features
    selected_features = []  # Initialize the list of selected features
    best_aic = float('inf')  # Initialize the best AIC value

    while features:
        candidates = [
            (sm.OLS(y, sm.add_constant(X[selected_features + [f]])).fit().bic, f) # # Calculate BIC for each candidate feature (can use the aic criterion)
            for f in features
            ]
        candidates.sort()  # Sort candidates based on criterion
        best_candidate_aic, best_candidate = candidates[0]  # Get the candidate with the lowest value

        if best_candidate_aic < best_aic:
            print(f"Adding feature {best_candidate} AIC/BIC {best_candidate_aic:.2f}") # Print the added feature and its AIC/BIC
            best_aic = best_candidate_aic  # Update the best AIC value
            selected_features.append(best_candidate)  # Add the selected feature to the list
            features.remove(best_candidate)  # Remove the selected feature from the candidate list
        else:
            break  # Stop if no further improvement in AIC/BIC
    return selected_features  # Return the list of selected features

In [ ]:
# running the stepwise function
selected_features = stepwise_selection(X_features, y)
print("Selected features:", selected_features)

In [ ]:
# Fitting to the selected features (get the features from X0 dataframe)
X2 = X0[selected_features + ['const']]  # Create a new feature matrix with selected features and constant term
model2 = sm.OLS(y, X2).fit()  # Fit an OLS regression model using selected features
print(model2.summary(alpha = 0.05))

In [ ]:
# Drawing the sscatterplot y-y_predict
y2_predict = model2.predict(X2)
plot_scatter(y2_predict, y)

In [ ]:
# Residuals Box-Plot
u_std_df['residuals_2'] = y-y2_predict  # Add the residuals to the unique explosives dataframe 
plot_residuals_by_class(u_std_df, residuals_col='residuals_2', group_col='group')

In [ ]:
# Checking variance inflation factor to assess multicollinearity
pd.Series([variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])], index=X2.columns)

In [ ]:
# Removing some features and Fitting the new model
X3=X2.drop(['NOO'], axis=1)
model3 = sm.OLS(y, X3).fit()
print(model3.summary(alpha = 0.05))

In [ ]:
# Checking variance inflation factor to assess multicollinearity
pd.Series([variance_inflation_factor(X3.values, i) for i in range(X3.shape[1])], index=X3.columns)

In [ ]:
# Drawing the sscatterplot y-y_predict
y3_predict = model3.predict(X3)
plot_scatter(y3_predict, y)

In [ ]:
# Residuals Box-Plot
u_std_df['residuals_3'] = y-y3_predict  # Add the residuals to the unique explosives dataframe 
plot_residuals_by_class(u_std_df, residuals_col='residuals_3', group_col='group')

In [ ]:
# ----Checking the model with non-aggregated data from the standardized database
std_df[target] = num_df[target] # Change the target to non-standardized value
std_df['num_type'] =  num_df['num_type']  # Change the num_type to non-standardized value
std_df['const'] = 1  # Add the model const

In [ ]:
# Drawing the sscatterplot y-y_predict (data not aggregated)
y3_predict_all = model3.predict(std_df[X3.columns])
plot_scatter(y3_predict_all, std_df[target])

In [ ]:
# Residuals Box-Plot
std_df['all_residuals_3'] = std_df[target] - y3_predict_all  # Add the residuals to the unique explosives dataframe 
plot_residuals_by_class(std_df, residuals_col='all_residuals_3', group_col='group')

## More Diagnostics Plots

The following code cell is from: https://www.statsmodels.org/dev/examples/notebooks/generated/linear_regression_diagnostics_plots.html
(Authors: Prajwal Kafle and Matt Spinelli)

In [ ]:
# ----base code
import statsmodels
import statsmodels.formula.api as smf
#import numpy as np
#import seaborn as sns
from statsmodels.tools.tools import maybe_unwrap_results
from statsmodels.graphics.gofplots import ProbPlot
#from statsmodels.stats.outliers_influence import variance_inflation_factor
#import matplotlib.pyplot as plt
from typing import Type

style_talk = 'seaborn-talk'    #refer to plt.style.available

class LinearRegDiagnostic():
    """
    Diagnostic plots to identify potential problems in a linear regression fit.
    Mainly,
        a. non-linearity of data
        b. Correlation of error terms
        c. non-constant variance
        d. outliers
        e. high-leverage points
        f. collinearity

    Authors:
        Prajwal Kafle (p33ajkafle@gmail.com, where 3 = r)
        Does not come with any sort of warranty.
        Please test the code one your end before using.

        Matt Spinelli (m3spinelli@gmail.com, where 3 = r)
        (1) Fixed incorrect annotation of the top most extreme residuals in
            the Residuals vs Fitted and, especially, the Normal Q-Q plots.
        (2) Changed Residuals vs Leverage plot to match closer the y-axis
            range shown in the equivalent plot in the R package ggfortify.
        (3) Added horizontal line at y=0 in Residuals vs Leverage plot to
            match the plots in R package ggfortify and base R.
        (4) Added option for placing a vertical guideline on the Residuals
            vs Leverage plot using the rule of thumb of h = 2p/n to denote
            high leverage (high_leverage_threshold=True).
        (5) Added two more ways to compute the Cook's Distance (D) threshold:
            * 'baseR': D > 1 and D > 0.5 (default)
            * 'convention': D > 4/n
            * 'dof': D > 4 / (n - k - 1)
        (6) Fixed class name to conform to Pascal casing convention
        (7) Fixed Residuals vs Leverage legend to work with loc='best'
    """

    def __init__(self,
                 results: Type[statsmodels.regression.linear_model.RegressionResultsWrapper]) -> None:
        """
        For a linear regression model, generates following diagnostic plots:

        a. residual
        b. qq
        c. scale location and
        d. leverage

        and a table

        e. vif

        Args:
            results (Type[statsmodels.regression.linear_model.RegressionResultsWrapper]):
                must be instance of statsmodels.regression.linear_model object

        Raises:
            TypeError: if instance does not belong to above object

        Example:
        >>> import numpy as np
        >>> import pandas as pd
        >>> import statsmodels.formula.api as smf
        >>> x = np.linspace(-np.pi, np.pi, 100)
        >>> y = 3*x + 8 + np.random.normal(0,1, 100)
        >>> df = pd.DataFrame({'x':x, 'y':y})
        >>> res = smf.ols(formula= "y ~ x", data=df).fit()
        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls(plot_context="seaborn-v0_8-paper")

        In case you do not need all plots you can also independently make an individual plot/table
        in following ways

        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls.residual_plot()
        >>> cls.qq_plot()
        >>> cls.scale_location_plot()
        >>> cls.leverage_plot()
        >>> cls.vif_table()
        """

        if isinstance(results, statsmodels.regression.linear_model.RegressionResultsWrapper) is False:
            raise TypeError("result must be instance of statsmodels.regression.linear_model.RegressionResultsWrapper object")

        self.results = maybe_unwrap_results(results)

        self.y_true = self.results.model.endog
        self.y_predict = self.results.fittedvalues
        self.xvar = self.results.model.exog
        self.xvar_names = self.results.model.exog_names

        self.residual = np.array(self.results.resid)
        influence = self.results.get_influence()
        self.residual_norm = influence.resid_studentized_internal
        self.leverage = influence.hat_matrix_diag
        self.cooks_distance = influence.cooks_distance[0]
        self.nparams = len(self.results.params)
        self.nresids = len(self.residual_norm)

    def __call__(self, plot_context='seaborn-v0_8-paper', **kwargs):
        # print(plt.style.available)
        # GH#9157
        if plot_context not in plt.style.available:
            plot_context = 'default'
        with plt.style.context(plot_context):
            fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
            self.residual_plot(ax=ax[0,0])
            self.qq_plot(ax=ax[0,1])
            self.scale_location_plot(ax=ax[1,0])
            self.leverage_plot(
                ax=ax[1,1],
                high_leverage_threshold = kwargs.get('high_leverage_threshold'),
                cooks_threshold = kwargs.get('cooks_threshold'))
            plt.show()

        return self.vif_table(), fig, ax,

    def residual_plot(self, ax=None):
        """
        Residual vs Fitted Plot

        Graphical tool to identify non-linearity.
        (Roughly) Horizontal red line is an indicator that the residual has a linear pattern
        """
        if ax is None:
            fig, ax = plt.subplots()

        sns.residplot(
            x=self.y_predict,
            y=self.residual,
            lowess=True,
            scatter_kws={'alpha': 0.5},
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        residual_abs = np.abs(self.residual)
        abs_resid = np.flip(np.argsort(residual_abs), 0)
        abs_resid_top_3 = abs_resid[:3]
        for i in abs_resid_top_3:
            ax.annotate(
                i,
                xy=(self.y_predict[i], self.residual[i]),
                color='C3')

        ax.set_title('Residuals vs Fitted', fontweight="bold")
        ax.set_xlabel('Fitted values')
        ax.set_ylabel('Residuals')
        return ax

    def qq_plot(self, ax=None):
        """
        Standarized Residual vs Theoretical Quantile plot

        Used to visually check if residuals are normally distributed.
        Points spread along the diagonal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        QQ = ProbPlot(self.residual_norm)
        fig = QQ.qqplot(line='45', alpha=0.5, lw=1, ax=ax)

        # annotations
        abs_norm_resid = np.flip(np.argsort(np.abs(self.residual_norm)), 0)
        abs_norm_resid_top_3 = abs_norm_resid[:3]
        for i, x, y in self.__qq_top_resid(QQ.theoretical_quantiles, abs_norm_resid_top_3):
            ax.annotate(
                i,
                xy=(x, y),
                ha='right',
                color='C3')

        ax.set_title('Normal Q-Q', fontweight="bold")
        ax.set_xlabel('Theoretical Quantiles')
        ax.set_ylabel('Standardized Residuals')
        return ax

    def scale_location_plot(self, ax=None):
        """
        Sqrt(Standarized Residual) vs Fitted values plot

        Used to check homoscedasticity of the residuals.
        Horizontal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        residual_norm_abs_sqrt = np.sqrt(np.abs(self.residual_norm))

        ax.scatter(self.y_predict, residual_norm_abs_sqrt, alpha=0.5);
        sns.regplot(
            x=self.y_predict,
            y=residual_norm_abs_sqrt,
            scatter=False, ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        abs_sq_norm_resid = np.flip(np.argsort(residual_norm_abs_sqrt), 0)
        abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]
        for i in abs_sq_norm_resid_top_3:
            ax.annotate(
                i,
                xy=(self.y_predict[i], residual_norm_abs_sqrt[i]),
                color='C3')

        ax.set_title('Scale-Location', fontweight="bold")
        ax.set_xlabel('Fitted values')
        ax.set_ylabel(r'$\sqrt{|\mathrm{Standardized\ Residuals}|}$');
        return ax

    def leverage_plot(self, ax=None, high_leverage_threshold=False, cooks_threshold='baseR'):
        """
        Residual vs Leverage plot

        Points falling outside Cook's distance curves are considered observation that can sway the fit
        aka are influential.
        Good to have none outside the curves.
        """
        if ax is None:
            fig, ax = plt.subplots()

        ax.scatter(
            self.leverage,
            self.residual_norm,
            alpha=0.5);

        sns.regplot(
            x=self.leverage,
            y=self.residual_norm,
            scatter=False,
            ci=False,
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
            ax=ax)

        # annotations
        leverage_top_3 = np.flip(np.argsort(self.cooks_distance), 0)[:3]
        for i in leverage_top_3:
            ax.annotate(
                i,
                xy=(self.leverage[i], self.residual_norm[i]),
                color = 'C3')

        factors = []
        if cooks_threshold == 'baseR' or cooks_threshold is None:
            factors = [1, 0.5]
        elif cooks_threshold == 'convention':
            factors = [4/self.nresids]
        elif cooks_threshold == 'dof':
            factors = [4/ (self.nresids - self.nparams)]
        else:
            raise ValueError("threshold_method must be one of the following: 'convention', 'dof', or 'baseR' (default)")
        for i, factor in enumerate(factors):
            label = "Cook's distance" if i == 0 else None
            xtemp, ytemp = self.__cooks_dist_line(factor)
            ax.plot(xtemp, ytemp, label=label, lw=1.25, ls='--', color='red')
            ax.plot(xtemp, np.negative(ytemp), lw=1.25, ls='--', color='red')

        if high_leverage_threshold:
            high_leverage = 2 * self.nparams / self.nresids
            if max(self.leverage) > high_leverage:
                ax.axvline(high_leverage, label='High leverage', ls='-.', color='purple', lw=1)

        ax.axhline(0, ls='dotted', color='black', lw=1.25)
        ax.set_xlim(0, max(self.leverage)+0.01)
        ax.set_ylim(min(self.residual_norm)-0.1, max(self.residual_norm)+0.1)
        ax.set_title('Residuals vs Leverage', fontweight="bold")
        ax.set_xlabel('Leverage')
        ax.set_ylabel('Standardized Residuals')
        plt.legend(loc='best')
        return ax

    def vif_table(self):
        """
        VIF table

        VIF, the variance inflation factor, is a measure of multicollinearity.
        VIF > 5 for a variable indicates that it is highly collinear with the
        other input variables.
        """
        vif_df = pd.DataFrame()
        vif_df["Features"] = self.xvar_names
        vif_df["VIF Factor"] = [variance_inflation_factor(self.xvar, i) for i in range(self.xvar.shape[1])]

        return (vif_df
                .sort_values("VIF Factor")
                .round(2))


    def __cooks_dist_line(self, factor):
        """
        Helper function for plotting Cook's distance curves
        """
        p = self.nparams
        formula = lambda x: np.sqrt((factor * p * (1 - x)) / x)
        x = np.linspace(0.001, max(self.leverage), 50)
        y = formula(x)
        return x, y


    def __qq_top_resid(self, quantiles, top_residual_indices):
        """
        Helper generator function yielding the index and coordinates
        """
        offset = 0
        quant_index = 0
        previous_is_negative = None
        for resid_index in top_residual_indices:
            y = self.residual_norm[resid_index]
            is_negative = y < 0
            if previous_is_negative == None or previous_is_negative == is_negative:
                offset += 1
            else:
                quant_index -= offset
            x = quantiles[quant_index] if is_negative else np.flip(quantiles, 0)[quant_index]
            quant_index += 1
            previous_is_negative = is_negative
            yield resid_index, x, y

In [ ]:
# Generating diagnostic plots
cls = LinearRegDiagnostic(model3)
# Residual vs Fitted values to identify non-linearity
cls.residual_plot()

In [ ]:
# Standarized Residual vs Theoretical Quantile to check if residuals are normally distributed
cls.qq_plot()

In [ ]:
# ---Sqrt(Standarized Residual) vs Fitted values to check homoscedasticity of the residuals
cls.scale_location_plot();

In [ ]:
# ----Residual vs Leverage to check observations that can sway the fit aka are influential (points falling outside the Cook’s distance curves)
cls.leverage_plot()

In [ ]:
# ----Cook’s distance curves can be drawn using other conventions (see above the code)
cls.leverage_plot(high_leverage_threshold=True, cooks_threshold='dof')

In [ ]:
# ----Checking variance inflation factor to assess multicollinearity
pd.Series([variance_inflation_factor(X3.values, i) for i in range(X3.shape[1])], index=X3.columns)

## Other packages: r-car, pingouin...
Should you wish to experiment with other statistical packages, you are at liberty to do so in order to tailor the utilities to your needs. Example:

conda install -c conda-forge pingouin

In [ ]:
# ----Load the package
import pingouin as pg

In [ ]:
# pip install pingouin

In [ ]:
# ----Pearson’s correlation
print('Pearson correlation')
p_corr = pg.corr(X_features['q_per_g'].values, y.values)
print(p_corr)
print('...............................................')
print('robust correlation')
r_corr = pg.corr(X_features['q_per_g'].values, y.values, method="bicor")
print(r_corr)

In [ ]:
# ----Testing the normality
# First get the residuals fro them last model
residuals3 = pd.DataFrame(y - model3.predict(X3))
# Numerical
print(pg.normality(residuals3, method='shapiro', alpha=0.05))
# Graphical with confidence curves
ax = pg.qqplot(residuals3, dist='norm', confidence=0.95)

# .....................................................................................................................<br>
## .....................................................................................................................<br>
### .....................................................................................................................<br>

# Linear regression using scikit_learn and cross-validation (cv)

In [ ]:
# ----Load the necessary libraries
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, LassoLarsIC
from sklearn.model_selection import KFold, cross_val_score, train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.feature_selection import RFECV

# Instance to LinearRegression class
clf = LinearRegression()

# We are going to start the examples with recursive feature elimination method to select the important feaures
# Initial parameters
min_features = 2  # Minimum number of features to consider
cv = KFold(n_splits=10, shuffle=True, random_state=111166) # Parameters of cv
# Working with Train/test splitting
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=111166)

# Using the method RFECV for recursive feature elimination by cross-validation (rmse criteria)
rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="neg_root_mean_squared_error", # you can change the criteria for scoring: "neg_median_absolute_error" "r2"
    min_features_to_select=min_features,
    n_jobs=1,
)
rfecv.fit(X_train, y_train)
# Use dir(rfecv) for seeing the avaliable methods
print(f"Optimal number of features: {rfecv.n_features_}")
print(f'The selected features are: {rfecv.get_feature_names_out()}')

In [ ]:
# ----Checking and plotting results
try:
    cv_results = pd.DataFrame.from_dict(rfecv.cv_results_)  # This conversion doesnt work on all versions 
    x_=cv_results["n_features"]
    y_=cv_results["mean_test_score"]
    yerr=cv_results["std_test_score"]
except Exception as e:
    x_=rfecv.cv_results_["n_features"]
    y_=rfecv.cv_results_["mean_test_score"]
    yerr=rfecv.cv_results_["std_test_score"]

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Mean test score")
plt.errorbar(
    x=x_,
    y=y_,
    yerr=yerr,
)
plt.title("Recursive Feature Elimination \n")
plt.show()
# rfecv.cv_results_

In [ ]:
# ----Print Features selected
print(rfecv.ranking_)
selected_features = X_features.columns[rfecv.support_]
print(selected_features)

# Comparing results

In [ ]:
# ----Funtion to calculate rmse from cv_test (train) and test: Linear Regression
def rmse_regression(features, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test):
    """
    Calculates and prints the RMSE for linear regression using given features.

    Args:
        features (list): A list of feature names to use in the regression.
        X_train (pandas.DataFrame, optional): The training feature matrix. Defaults to the global 'X_train' variable.
        y_train (pandas.Series, optional): The training target variable. Defaults to the global 'y_train' variable.
        X_test (pandas.DataFrame, optional): The testing feature matrix. Defaults to the global 'X_test' variable.
        y_test (pandas.Series, optional): The testing target variable. Defaults to the global 'y_test' variable.
    """
    for i in features:
        X_train = X_train[features]  # Select features for training
        X_test = X_test[features]  # Select features for testing
        lin_reg = LinearRegression().fit(X_train, y_train)  # Fit a linear regression model
        # Calculate rmse on training and test data using cross-validation
        lin_score_train = -1 * cross_val_score(lin_reg, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error').mean()
        lin_score_test = root_mean_squared_error(y_test, lin_reg.predict(X_test))
    print(f' cv_test_rmse and test_rmse for {features}: {lin_score_train:.2f}, {lin_score_test:.2f}')

In [ ]:
# ----The results of linear regression
rmse_regression(selected_features)  # Calculate and print RMSE for selected features
rmse_regression(X3.columns.drop('const'))  # Calculate and print RMSE for features in X3 (you have to drop the 'const')
rmse_regression(X2.columns.drop('const'))  # Calculate and print RMSE for features in X2 (you have to drop the 'const')

## You can try other regression techniques...
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html

In [ ]:
# ----rmse train and test: Lasso Regression
lasso_reg = LassoCV(max_iter=2000, random_state=111166, tol=0.012).fit(X_train, y_train) # Note that max_iter and tol values are set manually in this example
lasso_score_train = -1 * cross_val_score(lasso_reg, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error').mean()
lasso_score_test = root_mean_squared_error(y_test, lasso_reg.predict(X_test))
print(f'rmse cv_test and rmse test with Lasso {lasso_score_train:.2f}, {lasso_score_test:.2f}')
print(lasso_reg.coef_)
features_arg = X_train.columns[np.abs(lasso_reg.coef_)>0]
print(f'the selected features by Lasso are {[i for i in features_arg]}')

In [ ]:
#  ----Selecting Lasso model via an information criterion (selection of the alpha parameter value using the aic criterion)
lasso_lars_aic = LassoLarsIC(criterion="aic").fit(X_train, y_train)
# saving some results in a dataframe
results = pd.DataFrame(
    {
        "alphas": lasso_lars_aic.alphas_,
        "AIC criterion": lasso_lars_aic.criterion_,
    }
).set_index("alphas")
alpha_aic = lasso_lars_aic.alpha_
# run: dir(lasso_lars_aic) to see the methods

In [ ]:
# ----Printing the aic vs alpha
def highlight_min(x):
    """
    Highlights the minimum value in a Series.
    Args:
        x (pandas.Series): The input Series.
    Returns:
        list: A list of styles for each value in the Series.
    """
    x_min = x.min()
    return ["font-weight: bold" if v == x_min else "" for v in x]
results.style.apply(highlight_min)

In [ ]:
# ----Graphic results
ax = results.plot()
ax.vlines(
    alpha_aic,
    results["AIC criterion"].min(),
    results["AIC criterion"].max(),
    label="alpha: AIC estimate",
    linestyles="--",
    color="tab:blue",
)
ax.set_xlabel(r"$\alpha$")
ax.set_ylabel("criterion")
ax.set_xscale("log")
ax.legend()
_ = ax.set_title(
    f"Information-criterion for model selection"
)

In [ ]:
# ----Selected features by Lasso
features_Lasso_aic = X_train.columns[np.abs(lasso_lars_aic.coef_)>0]
print(f'the selected features by Lasso with the aic criterion are: {[i for i in features_Lasso_aic]}')

In [ ]:
# ----The results of linear regression with the Lasso_aic parameters
rmse_regression(features_Lasso_aic)

In [ ]:
# ----rmse train and test: Ridge Regression
ridge_reg = RidgeCV().fit(X_train, y_train)
ridge_score_train = -1 * cross_val_score(ridge_reg, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error').mean()
ridge_score_test = root_mean_squared_error(y_test, ridge_reg.predict(X_test))
print(f'rmse cv_test and rmse test with Ridge {ridge_score_train:.2f}, {ridge_score_test:.2f}')
print(ridge_reg.coef_)
threshold_weight = 0.01
features_Ridge = X_train.columns[np.abs(ridge_reg.coef_)>threshold_weight]
print(f'the selected features by Ridge are {[i for i in features_Ridge]}')
print('.................................................................')
print('.................................................................')
print('.................................................................')

## Principal Component Regression (PCR) and Cross-Validation
https://nirpyresearch.com/principal-component-regression-python/<br>
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html<br>
https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
# ----Trying PCA analysis for regression of principal components
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.model_selection import KFold, cross_val_score, train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.decomposition import PCA

# X_train, X_test, y_train, y_test = train_test_split(X_not_intercept, y, test_size=0.2, random_state=111166)

In [ ]:
# ----Calculate the principal components
pca = PCA() # default number of components = min(n_samples, n_features)
X_pc = pca.fit_transform(X_features)
print(X_pc)
print(X_pc.shape)

In [ ]:
# ----Heatmap for checking no correlation betwen principal components
X_pc_df = pd.DataFrame(X_pc, columns=[f'PC{i}' for i in range(0, len(X_features.columns))])
pca_correlation_matrix = X_pc_df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(pca_correlation_matrix, annot=True, cmap='coolwarm')
plt.title('PCs Heatmap')
plt.show()

In [ ]:
# ----Showing features weights of each principal component
loadings = pd.DataFrame(pca.components_.T, columns=X_pc_df.columns, index=X_features.columns[:])
print(loadings)

In [ ]:
# ----Heatmap for the loadings (same information in a visual way)
plt.figure(figsize=(12, 8))
sns.heatmap(loadings, annot=True, cmap='coolwarm')
plt.title('PCA Loadings Heatmap')
plt.show()

In [ ]:
# ----Checking the most important features for each principal component
# Choose a threshold for which features are considerated important for each principal component
threshold = 0.3

# Find features with loadings above the threshold for each principal component
important_features = {}
for column in loadings.columns:
    important_features[column] = loadings.index[loadings[column].abs() > threshold].tolist()

# Show the important features for each principal component
for pc, features in important_features.items():
    print(f"{pc}: {', '.join(features)}")

In [ ]:
# ----Explained variance of each principal component
print([f"{i:.1%}" for i in pca.explained_variance_ratio_])

In [ ]:
# ----Working with Train/test splitting
X_train, X_test, y_train, y_test = train_test_split(X_pc, y, test_size=0.2, random_state=111166)

In [ ]:
# -----Calculate the r2 variation, using KFold, with the number of components used in the regression
# Define cross-validation folds
cv = KFold(n_splits=10, shuffle=True, random_state=111166)

# Linear regression instance
lin_reg = LinearRegression()

# Store RMSE for each regression case
rmse_store = []

# Loop through principal components for linear regression
for i in range(1, X_train.shape[1]+1):
    rmse_ = cross_val_score(lin_reg,
                            X_train[:,:i], # Use first k principal components
                            y_train,
                            cv=cv,
                            scoring='neg_root_mean_squared_error')
    rmse_store.append(rmse_)
rmse_mean = [np.mean(i) for i in rmse_store]
rmse_std = [np.std(i) for i in rmse_store]
print('Number of principal components:')
print([i for i in range(1, X_train.shape[1]+1)])
print('rmse_mean')
print([f"{i:.2f}" for i in rmse_mean])
print('rmse_std')
print([f"{i:.2f}" for i in rmse_std])

In [ ]:
# ----Looking for the best option
plt.plot(rmse_mean, '-x')
plt.xlabel('Number of principal components')
plt.ylabel('rmse[test_from_cv]')
plt.xticks(np.arange(X_train.shape[1]), np.arange(1, X_train.shape[1]+1))
plt.axhline(y=np.max(rmse_mean), color='r', linestyle='-');

In [ ]:
# ----Another method to calculate the r2 and rmse variation, using KFold, with the number of components used in the regression
# Define cross-validation folds
cv = KFold(n_splits=10, shuffle=True, random_state=111166)

# Linear regression instance
lin_reg = LinearRegression()

# Store RMSE for each regression case
fit_meas = []

# Loop through principal components for linear regression
for i in range(1, X_train.shape[1]+1):
    measures = cross_validate(lin_reg,
                                X_train[:,:i], # Use first k principal components
                                y_train,
                                cv=cv,
                                scoring=('r2', 'neg_root_mean_squared_error'))
    fit_meas.append(measures)

In [ ]:
# ----See the contents of cross_validate object
print([i for i in fit_meas[0]])

In [ ]:
# ----Pass the data to dataframe
fit_meas_df = pd.DataFrame(fit_meas)
print('Dataframe 0 dimension = number of principal components: ', fit_meas_df.shape[0])
print('r2[test]', [fit_meas_df['test_r2'][i].mean() for i in range(fit_meas_df.shape[0])])
print('rmse[test]', [-1*fit_meas_df['test_neg_root_mean_squared_error'][i].mean() for i in range(fit_meas_df.shape[0])])


In [ ]:
# ----Looking for the best option
plt.plot([fit_meas_df['test_r2'][i].mean() for i in range(fit_meas_df.shape[0])], '-x')
plt.xlabel('Number of principal components')
plt.ylabel('r2[test_from_crosss_validation]')
plt.xticks(np.arange(X_train.shape[1]), np.arange(1, X_train.shape[1]+1));

In [ ]:
# ----Selecting the number of principal components
pc_number = 5
X_train = X_train[:,:pc_number]
X_test = X_test[:,:pc_number]
print(f' Selected {pc_number} components, X_train[0:5,:]')
print(X_train[0:5,:])
print(' ....................')

In [ ]:
# ----rmse train and test: Linear Regression
lin_reg = LinearRegression().fit(X_train, y_train)
lin_score_train = -1 * cross_val_score(lin_reg, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error').mean()
lin_score_test = root_mean_squared_error(y_test, lin_reg.predict(X_test))
print(f' cv_test_rmse and test_rmse: {lin_score_train:.2f}, {lin_score_test:.2f}')

In [ ]:
# ----r2 score without cv (only pc_number components)
print(f' train_r2 and test_r2 with {pc_number} components: {lin_reg.score(X_train, y_train):.2%}, {lin_reg.score(X_test, y_test):.2%}')

In [ ]:
# ----Drawing the sscatterplot y-y_predict
plot_scatter(lin_reg.predict(X_test), y_test)

In [ ]:
# ----r2 using the r2_score function
print(f'r2_train: {r2_score(y_train, lin_reg.predict(X_train)):.2%}')

In [ ]:
# ----Linear regression using all components
lin_reg_all = LinearRegression().fit(np.array(X_pc), y)
y_predict =  lin_reg_all.predict(np.array(X_pc))
print(f'r2_all_components_all_data {r2_score(y, y_predict):.2%}')
print(f'the rmse with all data and components is: {root_mean_squared_error(y, lin_reg_all.predict(X_pc)): .2f}')

## Partial Least Squares (PLS) regression
https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_predict

In [ ]:
# ----As with PCA, you can use all the features or only the most correlated ones
n_components = X_features.shape[1] # all the components in this example
pls_all = PLSRegression(n_components=n_components)
pls_all.fit(X_features, y) # PLS scale the features
y_predict = pls_all.predict(X_features)
print('number of components: ', n_components)
print(f"PLS r-squared including all data and components in the regression: {pls_all.score(X_features, y):.2%}")
print(f"and the rmse is {np.sqrt(mean_squared_error(y, y_predict)):.2f}")

In [ ]:
# ----Get the most important features: those with the largest absolute values of the coefficients of the regression (y = coefficients X + intercept)
print('the features analysed are ', pls_all.feature_names_in_)
print('the coefficients of the linear model are', pls_all.coef_, 'and the intercept is', pls_all.intercept_)

In [ ]:
# ----PLS regression with cross-validation: looking for the best number of components
parameters = {'n_components': np.arange(1, X_features.shape[1],1)}
pls = GridSearchCV(PLSRegression(), parameters, cv=cv, scoring = 'neg_root_mean_squared_error')
pls.fit(X_features, y)
print('the best number of components is ', pls.best_estimator_)
print(f'the best score (cv_test_rmse) is, {-1*pls.best_score_: .2f}')
print(f'the features analysed are ', pls.feature_names_in_)

In [ ]:
# ----Showing and checking results
results = pls.cv_results_
results = pd.DataFrame(results)
print(results)

In [ ]:
# ----Looking for the best option
plt.plot([-1*results['mean_test_score'][i] for i in range(results.shape[0])], '-x')
plt.xlabel('Number of components')
plt.ylabel('rmse[test_from_cv]')
plt.xticks(np.arange(results.shape[1]), np.arange(1, results.shape[1]+1))
plt.axhline(y=-1*results['mean_test_score'].max(), color='r', linestyle='-');

In [ ]:
# ----Checking the r2 for the best estimator
y_cv = cross_val_predict(pls.best_estimator_, X_features, y, cv=cv)
rmse_pls, score_pls = np.sqrt(mean_squared_error(y, y_cv)), r2_score(y, y_cv)
print(f'the cv_rmse is: {rmse_pls:.2f}, and the cv_r2 is {score_pls:.2%}')

In [ ]:
# ----Funtion to calculate the best estimator by ssquential features/components elimination
def variable_elimination_pls(X, y, feature_names, initial_n_components, cv=10):
    """
    Performs sequential variable and component elimination,
    storing all solutions in a DataFrame.

    Args:
        X (numpy.ndarray): Matrix of independent variables (n_samples, n_features).
        y (numpy.ndarray): Vector of the dependent variable (n_samples,).
        feature_names (list): List of feature names (length: n_features).
        initial_n_components (int): Initial number of PLS components.
        cv (int, optional): Number of folds for cross-validation. Defaults to 10.

    Returns:
        pandas.DataFrame: DataFrame containing the results of each iteration,
                          with the following columns:
                          - 'n_components': Number of PLS components.
                          - 'variables_indices': Indices of the remaining variables.
                          - 'variables_names': Names of the remaining variables.
                          - 'score': Score (neg_root_mean_squared_error) of the iteration.
    """

    # Check that the length of feature_names matches the number of columns in X
    if len(feature_names) != X.shape[1]:
        raise ValueError("The length of 'feature_names' must be equal to the number of columns in 'X'")

    current_n_components = initial_n_components
    remaining_vars = list(range(X.shape[1]))
    print(remaining_vars)
    # List to store the results of each iteration
    results = []

    while len(remaining_vars) > 0:
        X_subset = X[:, remaining_vars]
        best_score = float('inf')  # Initialize with the worst possible score
        best_n_components = 0

        while current_n_components > 0:
            pls = PLSRegression(n_components=current_n_components)
            scores = cross_val_score(pls, X_subset, y, cv=cv, scoring='neg_root_mean_squared_error')
            mean_score = np.mean(scores)

            # Store the results of this iteration in the list
            results.append({
                'n_features': X_subset.shape[1],
                'n_components': current_n_components,
                'variables_indices': [i for i in remaining_vars],
                'variables_names': [feature_names[i] for i in remaining_vars],
                'score': -1 * mean_score
            })

            # Update the best score and number of components
            if mean_score < best_score:
                best_score = mean_score
                best_n_components = current_n_components

            current_n_components -= 1

        current_n_components = X_subset.shape[1] - 1
        if len(remaining_vars) > 1:
            pls = PLSRegression(n_components=best_n_components)  # Use the best n_components
            pls.fit(X_subset, y)
            coefficients = pls.coef_.flatten()
            least_important_var_index = np.argmin(np.abs(coefficients))
            del remaining_vars[least_important_var_index]
        else:
            break

    # Create a DataFrame from the list of results
    df_results = pd.DataFrame(results)
    return df_results

In [ ]:
# ----Getting the results of the features elimination
df_results = variable_elimination_pls(X_features.values, y.values, X_features.columns, X_features.shape[1], cv=cv)
print(df_results)

In [ ]:
# ----Show the result with the best score
best_result_index = df_results['score'].idxmin()
best_result = df_results.loc[best_result_index]
print("\nBest result:")
print(best_result.to_markdown(numalign="left", stralign="left"))

In [ ]:
# ----Checking anothers solutions
bests_results = df_results.query('score<0.30 & n_features<4')  # Try with some values
print(bests_results.to_markdown(numalign="left", stralign="left"))

## Random Forest Regression
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

# Split data into training and testing sets
X = X_features[['Oxy_Balance', 'Moment1', 'q_per_g', 'num_type']]  # Select features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111166)

# Define the hyperparameter grid to search the optimal values
param_grid = {
    'n_estimators': [10, 30, 50],
    'max_depth': [3, 5, 7],
    'min_samples_split': [4, 6, 8],
    'min_samples_leaf': [4, 6, 8]
}

# Create a Random Forest model
rf_model = RandomForestRegressor(random_state=111166, oob_score=True)

# Negative root mean squared error
neg_rmse = make_scorer(root_mean_squared_error, greater_is_better=False)

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=cv, scoring=neg_rmse, verbose=2, n_jobs=-1)

# Fit the GridSearchCV to the training data
grid_search.fit(X_train, y_train)

In [ ]:
# ----Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)
print(f'the mean cross-validated score (rmse) of the best_estimator is {-1*grid_search.best_score_:0.3f}')

In [ ]:
# ----Get the best model
best_rf_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_rf_model.predict(X_test)

# Evaluate the model
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred))
r2_rf = r2_score(y_test, y_pred)
y_pred_train = best_rf_model.predict(X_train)
rmse_rf_train = np.sqrt(mean_squared_error(y_train, y_pred_train))

# Print the results
print(f'Best score (r2) train: {best_rf_model.score(X_train, y_train):.2%}')
print(f"R-squared (r2) test: {r2_rf:.2%}")
print(f"Root Mean Squared Error (rmse) train : {rmse_rf_train:.2f}")
print(f"Root Mean Squared Error (rmse) test: {rmse_rf:.2f}")

In [ ]:
# ----Check results
rf_results = pd.DataFrame(grid_search.cv_results_)
print(rf_results)

In [ ]:
# ----You can choose yours best parameters
my_best = 10
good_rf_params = grid_search.cv_results_['params'][my_best]
print(good_rf_params)
rf_model.set_params(**good_rf_params)
my_best_rf_model = rf_model.fit(X_train, y_train)

In [ ]:
# Display scatterplot of the prediction
y_pred = my_best_rf_model.predict(X_test)
# Drawing the sscatterplot y-y_predict
plot_scatter(y_pred, y_test)

In [ ]:
# OOB score (Out-of-Bag score)
if hasattr(my_best_rf_model, 'oob_score_') and my_best_rf_model.oob_score_:
    print(f'r2 OOB (Out-of-Bag): {my_best_rf_model.oob_score_:.2f}')
mse = mean_squared_error(y_test, y_pred)
print(f'the test_rsme is {np.sqrt(mse):.2f}')

In [ ]:
# Plot feature importances
importances = my_best_rf_model.feature_importances_
features = X.columns
indices = np.argsort(importances)

plt.figure(figsize=(10, 6))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
# Plot permutation importances
from sklearn.inspection import permutation_importance

# Plot the permutation importance
result = permutation_importance(
    my_best_rf_model, X_test, y_test, n_repeats=50, random_state=111162, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

# Model Output Explainability: SHAP
https://shap.readthedocs.io/en/latest/index.html

In [ ]:
# ---- Explain the model's predictions using SHAP
import shap
explainer = shap.Explainer(my_best_rf_model)
shap_values = explainer(X)

# Visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])

In [ ]:
# ---- Create a scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:, ["Oxy_Balance", 'num_type']], color=shap_values[:, 'q_per_g'])

In [ ]:
# ----Summarize the effects of all the features
shap.plots.beeswarm(shap_values)

In [ ]:
# ---- Mean absolute value of the SHAP values and absolute values for each feature
shap.plots.bar(shap_values)
shap.plots.beeswarm(shap_values.abs)

In [ ]:
# ---- Mean absolute value of the SHAP values for each group
group = ["NNOO" if shap_values[i, "num_type"].data == 0 else "CN0O" if shap_values[i, "num_type"].data == 1 else 'NOOO'
         for i in range(shap_values.shape[0])]
shap.plots.bar(shap_values.cohorts(group).abs.mean(axis=0))

In [ ]:
# ----Shapley values through KernelExplainer
# KernelExplainer estimates Shapley values by sampling subsets and comparing predictions with and without each feature (assumes independence)
background_data = shap.sample(X_train, 3000, random_state=111166)  # 3000 is the number of subsets
my_model = lambda x: my_best_rf_model.predict(pd.DataFrame(x, columns=list(X_train.columns)))
ker_expl = shap.KernelExplainer(my_model, background_data, link='identity')
shap_vals_ker = ker_expl(X_test.sample(len(X_test), replace=False, random_state=111166))

shap.plots.bar(shap_vals_ker)

In [ ]:
# ----Nw we are going to introduce the correlation between features
# Get the distance_correlations between features
clust = shap.utils.hclust(X_train, metric=distance_correlation)

In [ ]:
# ----Accounting for feature interactions - correlations
# Calculate Shapley values for feature coalitions, then distributing within each coalition
# The PartitionExplainer creates a binary-hierarchal clustering of features and then caculates the feature attribution with the marginals respecting this binary partition tree
masker = shap.maskers.Partition(X_train, clustering=clust)
part_expl = shap.PartitionExplainer(my_model, masker)  # Specify feature coalitions based on their interactions
shap_vals_part = part_expl(X_test.sample(len(X_test.values), random_state=111166))
shap.plots.bar(shap_vals_part, clustering_cutoff=0.7)

# Visualizing graphs with the graphviz library
1-Download an install graphviz form: https://graphviz.org/download/
2- Run in the EDA environment: conda install graphviz python-graphviz pydot

In [ ]:
# import the libraries
from sklearn.tree import export_graphviz
import graphviz

# Choose some tree (the 0 tree in this example)
choose_tree = 0
estimator_to_plot = my_best_rf_model.estimators_[choose_tree]
# Create the graph
dot_data = export_graphviz(estimator_to_plot,
                           out_file=None,
                           feature_names=X.columns,
                           filled=True,
                           rounded=True,
                           special_characters=True,
                           impurity=True,
                           max_depth=4)
graph = graphviz.Source(dot_data)
# Display the graph
display(graph)